In [4]:
import pandas as pd
import sqlite3
import requests
from io import StringIO
import xml.etree.ElementTree as ET

In [5]:
# URL der XML-Datei
xml_url = "https://raw.githubusercontent.com/Indorhc3/LetsMeet/main/Lets_Meet_Hobbies.xml"


In [6]:
# XML-Datei aus der URL herunterladen
response = requests.get(xml_url)
response.raise_for_status()  # Sicherstellen, dass der Download erfolgreich war

In [7]:
# XML-Daten einlesen und parsen
tree = ET.ElementTree(ET.fromstring(response.text))
root = tree.getroot()

In [8]:
# Liste zur Speicherung der extrahierten Daten
data = []

# XML-Daten durchlaufen und extrahieren
for entry in root.findall('entry'):  # Beispiel: Angenommen, jedes Element im XML hat den Tag 'entry'
    name = entry.find('name').text if entry.find('name') is not None else None
    hobby = entry.find('hobby').text if entry.find('hobby') is not None else None
    priority = entry.find('priority').text if entry.find('priority') is not None else None
    data.append((name, hobby, priority))

In [9]:

# Erstelle DataFrame aus den extrahierten Daten
df = pd.DataFrame(data, columns=['name', 'hobby', 'priority'])

In [10]:
# Verbindung zur SQLite-Datenbank herstellen (oder einer anderen Datenbank, falls gewünscht)
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

In [11]:
# Tabelle erstellen, falls nicht vorhanden
create_table_query = """
CREATE TABLE IF NOT EXISTS hobbies_from_xml (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    hobby TEXT,
    priority TEXT
);
"""
cursor.execute(create_table_query)

In [12]:
# Daten in die Datenbank einfügen
for _, row in df.iterrows():
    cursor.execute("""
    INSERT INTO hobbies_from_xml (name, hobby, priority)
    VALUES (?, ?, ?)
    """, (row['name'], row['hobby'], row['priority']))

In [13]:
# Änderungen speichern und Verbindung schließen
conn.commit()
conn.close()

print("Daten aus der XML-Datei wurden erfolgreich importiert.")

Daten aus der XML-Datei wurden erfolgreich importiert.
